In [75]:
import os
import csv
import sys
import urllib2
import pandas as pd
import PRData
import traceback
import watson_developer_cloud
import watson_developer_cloud.natural_language_understanding.features.v1 as features


#Credentials for access to Watson
credentials = {
  "url": "https://gateway.watsonplatform.net/natural-language-understanding/api",
  "username": "fba7b9c0-abbb-4d30-985d-0729614689b9",
  "password": "1I4nEsfDEUMs"
}

#Headers for urllib2 in use at process_url function
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}


nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2017-02-27',
                                                            username=credentials['username'],
                                                            password=credentials['password'])


####################
# Functions I am using in this transformation file, consider implications of these, their naming, and location

def find_uniques(input_list):
    """This function takes in an input_list, this is assumed to have duplicates,
    and outputs a list that is unique and has beginning and 
    trailing spaces removed"""
    input_list = input_list.tolist()
    uniques = []
    for l in input_list:
        temp = l.split(',')
        for t in temp:
            uniques.append(t.lstrip(' ').rstrip(' '))
    return uniques


def process_response(watson_response):
    # Here I am keeping what I need defined to this function. Consider this choice architecturally.
    text = []
    relevance = []

    for key in watson_response['entities']:
        text.append(key['text'])
        relevance.append(key['relevance'])   
    data = {'relevance':relevance,'text':text}
    dataframe = pd.DataFrame(data)

    return dataframe



# Tag the dataframe, i.e. create columns 'Jurisdiction_autotag' and 'Member_autotag'
def watson_tagging(dataframe):
    """Processes the dataframe from PRData.data_pull() and returns the dataframe
    with two additional columns Jurisdiction_autotag & Member_autotag."""
    for index, row in dataframe.iterrows():
        url = dataframe.ix[index]['URL'].lstrip().rstrip()
        members_in_text = []
        juris_in_text = []

        try:
            
            print url
            w_response = nlu.analyze(url=url,features=[features.Entities()])
            entities = process_response(w_response)
            print entities['text']
            
            for member in unique_members:
                for item in entities['text']:
                    if member in item:
                        members_in_text.append(member)

            for jurisdiction in unique_juris:
                for item in entities['text']:
                    if jurisdiction in item:
                        juris_in_text.append(jurisdiction)
                        
        except Exception:
            print(traceback.format_exc())
            print "Could not process: %s" % url

        # turn lists of jurisdictions and members into comma-separated string
        juris_in_text = ', '.join(juris_in_text)
        members_in_text = ', '.join(members_in_text)

        # If they exist, then assign the string or '-' for nothing
        if juris_in_text is not None:
            dataframe.ix[index]['Jurisdiction_autotag'] = juris_in_text
        else:
            dataframe.ix[index]['Jurisdiction_autotag'] = '-'

        # If they exist, then assign the string or '-' for nothing
        if members_in_text is not None:
            dataframe.ix[index]['Member_autotag'] = members_in_text
        else:
            dataframe.ix[index]['Member_autotag'] = '-'
        
    return dataframe



In [ ]:
####################
# Process the current data down and build what I need from it

current_month = 'July 2016' #MOVE TO CONFIG

data = PRData.data_pull()
current_data = data[data['DATE'] == current_month]

# This section is used to process all the data to end up with unique lists of jurisdictions & member
# this has to be ran before the next section. This needs to be moved as it's more of a helper function
# The data needs to be output and preserved and then called into this script
juris = data['Jurisdictions Mentioned'].unique()
members = data['Members Mentioned']

juris = find_uniques(juris)
members = find_uniques(members)

unique_juris = list(set(juris))
unique_juris = filter(None,unique_juris)
unique_members = list(set(members))
unique_members = filter(None, unique_members)

##########
# Using what is previous created, get started with watson request and handling
# Response



In [72]:
taggable_content = data[~data['URL'].str.contains('images')]
taggable_content = taggable_content[~taggable_content['TYPE'].str.contains('vision')]
taggable_content.shape

tagged_content = watson_tagging(taggable_content.head(3))

(629, 13)